In [1]:
import random
from duraksub import Card, NoCard, Hand, Deck, Pair, Discard, Board, BaseGame, BaseAction, BasePlayer

class Player(BasePlayer):
    def __init__(self):
        super(BasePlayer, self).__init__()
        self.hand = Hand()
        
class Action(BaseAction):
    def __init__(self, game, player, defender, verb, card=Card(), target=Card()):
        super(BaseAction, self).__init__()
        self.game = game
        self.player = player
        self.defender = defender
        self.verb = verb
        self.card = card
        self.target = target
        
    def randomize(self):
        self.player = random.choice(game.players)
        self.defender = random.choice(game.players)
        self.verb = random.choice(['pass', 'pickup', 'cover', 'play', 'reverse'])
        self.card = Card.random()
        self.target = Card.random()

class Game(BaseGame):
    def __init__(self):
        super(BaseGame, self).__init__()
        self.deck = Deck()
        self.board = Board()
        self.discard = Discard()
        self.players = [Player(), Player(), Player()]
        self.attacker = self.players[0]
        self.defender = self.players[1]
        self.trump = Card(0,0)
        
    def randomizeBoard(self):
        self.board = Board()
        while random.random() > 0.2:
            a = Card.random()
            b = Card.random() if random.random() > 0.5 else Card()
            self.board.append(Pair(a,b))
            
    def randomizeDiscard(self):
        self.discard = Discard()
        while random.random() > 0.2:
            self.discard.append(Card.random())
            
    def randomize(self):
        self.randomizeBoard()
        self.randomizeDiscard()
        self.trump = Card.random()
        
game = Game()
game.randomizeBoard()
game.randomizeDiscard()

print(game.board)
print(game.discard)

[[24, NoCard], [7, NoCard]]
[14]


In [18]:
from inspect import signature
from itertools import permutations
from functools import partial
from anytree import Node, NodeMixin, RenderTree, PreOrderIter
import duraksub
from duraksub import eq, allowType

class Query:
    def __init__(self, verb, params={}, arghints=[]):
        self.verb = verb
        self.params = params
        self.arghints = arghints
        
    def oneHot(self):
        return F.one_hot(torch.Tensor([queries.index(self)]).long().cuda(),num_classes=10).float()
    
    def compatible(self, arg):
        return any(allowType(arg, typ) for typ in self.arghints)
            
    def __hash__(self):
        return hash(str(self))
    
    def __eq__(self, other):
        return type(other) == Query and self.verb == other.verb
        
    def __repr__(self):
        return f'{self.verb}'

class MissingParamException(Exception):
    def __init__(self, names):
        super().__init__()
        self.names = names
        
    def __repr__(self):
        return f'Missing params {self.names}'
    
class MissingTargetException(Exception):
    def __init__(self):
        super().__init__()
        
    def __repr__(self):
        return 'Missing target and no learned sequence'
    
class NoneArgException(Exception):
    def __init__(self):
        super().__init__()
        
    def __repr__(self):
        return 'None in args, one of children failed'
    
class NotReadyException(Exception):
    def __init__(self):
        super().__init__()
        
    def __repr__(self):
        return 'Subroutine does not have all args'
    
class Sequence(NodeMixin):
    def __init__(self, func, query=None, parent=None, children=None, res=None, part=None):
        self.sig = signature(func)
        self.func = func
        self.query = query
        self.parent = parent
        if children is not None:
            self.children = children
        self.res = res
        self.partial = partial(func) if part is None else part
    
    def bind(self, params={}):
        bound = {}
        for name in self.sig.parameters:
            if name in params:
                bound[name] = params[name]
        if self.children:
            for child in self.children:
                child.bind(params)
        self.partial = partial(self.func, **bound)
    
    def clean(self):
        for child in self.children:
            child.clean()
        self.res = None
        self.partial = None
        
    def copy(self):
        return Sequence(self.func, self.query, None, [child.copy() for child in self.children], self.res, self.partial)
        
    def getUnbound(self):
        return set([name for name in self.sig.parameters]) - set(self.partial.keywords.keys())
        
    def ready(self):
        return len(self.partial.keywords) == len(self.sig.parameters)
    
    def __call__(self):
        if not self.children:
            if not self.ready():
                raise NotReadyException()
            self.res = self.partial()
        else:
            args = []
            for child in self.children:
                args.append(child())
            if None in args:
                raise NoneArgException()
            self.res = self.partial(*args)
        return self.res
    
    def __eq__(self, other):
        return (type(other) == Sequence and 
                self.func == other.func and 
                len(self.children) == len(other.children) and
                all(x == y for x,y in zip(self.children, other.children)))
    
    def __repr__(self):
        children = [child.func.__name__ for child in self.children]
        nBound = len(self.partial.keywords) if self.partial is not None else 0
        return f'{str(self.query)} {self.func.__name__} {len(self.sig.parameters)} {nBound} {children} {self.res}'

class DurakAI:
    def __init__(self):
        self.seqs = [Sequence(sub) for sub in duraksub.getFunctions(duraksub)]
        self.rules = []
        self.graveyard = []
        
    def __call__(self, query, target=None):
        for seq in self.seqs:
            if seq.query == query:
                seq.bind(query.params)
                res = None
                try:
                    res = seq()
                except Exception as ex:
                    pass
                except:
                    raise
                if target is not None and not eq(res, target):
                    self.invalidate(query)
                else:
                    return res
        
        # Need a target if we don't have a prepared seq
        if target is None:
            raise MissingTargetException()
        
        # Clean existing bindings
        for seq in self.seqs:
            seq.clean()
        
        # Bind query params
        for seq in self.seqs:
            seq.bind(query.params)
            
        # Invoke callable sequences
        for seq in self.seqs:
            res = None
            try:
                res = seq()
            except Exception as ex:
                pass
            except:
                raise
            if seq.query == query:
                if eq(res, target):
                    return res
                self.invalidate(query)
            elif eq(res, target):
                self.register(seq.copy(), query)
                return res
                
        leaves = [seq for seq in self.seqs if seq.res is not None]
        
        for i in range(3):
            for seq in self.seqs:
                n = len(seq.getUnbound())
                if n == 0:
                    continue
                    
                # Trim based on type hints
                hints = [typ for name, typ in seq.func.__annotations__.items() if name != 'return'] 
                if len(hints) > 0:
                    valid = [leaf for leaf in leaves if any(allowType(leaf.res, typ) for typ in hints)]
                    perms = list(permutations(valid, n))
                else:
                    perms = list(permutations(leaves, n))
                    
                print(seq)
                print(f'{n} {len(perms)} {len(leaves)}')
                for p in perms:
                    # Check validity of argument types
                    if not all(allowType(node.res, typ) for node, typ in zip(p, hints)):
                        continue
                    try:
                        cseq = seq.copy()
                        cseq.children = [node.copy() for node in p]
                        # Don't recreate tentative sequences
                        if cseq in self.seqs or cseq in leaves:
                            continue
                        # Known failed sequences
                        try:
                            idx = self.graveyard.index(cseq)
                            if self.graveyard[idx].query == query:
                                continue
                        except ValueError:
                            pass
                        res = cseq()
                        if res is None:
                            continue
                        if eq(res, target):
                            # Check query arghints
                            if len(query.arghints) > 0:
                                if not all(query.compatible(child.res) for child in cseq.children):
                                    continue
                            self.register(cseq, query)
                            return res
                        leaves.append(cseq)
                    except Exception as ex:
                        print(ex)
                    except KeyboardInterrupt:
                        print('KeyboardInterrupt')
                        print(f'{n} {len(perms)} {len(leaves)}')
                        raise
                    except:
                        raise
    
    def register(self, seq, query):
        self.seqs.append(seq)
        seq.query = query
        
    def invalidate(self, query):
        toRemove = []
        for seq in self.seqs:
            if seq.query == query:
                toRemove.append(seq)
        for seq in toRemove:
            print(f'Invalidated {seq.query.verb}')
            self.seqs.remove(seq)
            self.graveyard.append(seq)

def printTree(seq):
    for pre, _, node in RenderTree(seq):
        print(f'{pre}{node}')
        
ai = DurakAI()
                        
print('Complete')

Complete


In [33]:
from duraksub import *

game.randomize()
action = Action(game,0,1,'pickup')
action.randomize()

for seq in ai.seqs:
    seq.clean()
    printTree(seq)
    
ai(Query('get uncovered cards on board', {'game': game, 'action': action}), count(getUncovered(game.board)))
ai(Query('get num pairs on board', {'game': game}), count(game.board))
ai(Query('get num cards in discard', {'game': game}), count(game.discard))
if len(game.discard) > 1: # Faulty logic or planning: when shu
    ai(Query('3 beats a 2', {'game': game}, [Card]), beats(game.discard[0],game.discard[1],game.trump))
    ai(Query('2 beats a 3', {'game': game}, [Card]), beats(game.discard[1],game.discard[0],game.trump))

for seq in ai.seqs:
    printTree(seq)

None allSameRank 1 0 [] None
None beats 3 0 [] None
None contains 2 0 [] None
None count 1 0 [] None
None getAttacker 1 0 [] None
None getBoard 1 0 [] None
None getCardA 1 0 [] None
None getDefender 1 0 [] None
None getDefenderA 1 0 [] None
None getDiscard 1 0 [] None
None getHand 1 0 [] None
None getIndex 2 0 [] None
None getItem 2 0 [] None
None getNoCard 0 0 [] None
None getPlayerA 1 0 [] None
None getRank 1 0 [] None
None getSuit 1 0 [] None
None getTargetA 1 0 [] None
None getTrump 1 0 [] None
None getUncovered 1 0 [] None
None getVerbA 1 0 [] None
None hasRank 2 0 [] None
None isPositive 1 0 [] None
None isZero 1 0 [] None
None lessThan 2 0 [] None
None verbCheckFn 1 0 [] None
None verbCheckFn 1 0 [] None
None verbCheckFn 1 0 [] None
None verbCheckFn 1 0 [] None
None verbCheckFn 1 0 [] None
get num pairs on board count 1 0 ['getBoard'] None
└── None getBoard 1 0 [] None
get num cards in discard count 1 0 ['getDiscard'] None
└── None getDiscard 1 0 [] None
get uncovered cards on b

In [34]:
for poop in ai.graveyard:
    print(poop)

get uncovered cards on board count 1 0 ['getBoard'] 2
2 beats a 3 beats 3 0 ['getNoCard', 'getTrump', 'getItem'] False
2 beats a 3 beats 3 0 ['getNoCard', 'getTrump', 'getItem'] False
2 beats a 3 beats 3 0 ['getItem', 'getNoCard', 'getItem'] None
2 beats a 3 beats 3 0 ['getNoCard', 'getItem', 'getTrump'] False
2 beats a 3 beats 3 0 ['getItem', 'getNoCard', 'getItem'] False
2 beats a 3 beats 3 0 ['getTrump', 'getNoCard', 'getItem'] False
